In [4]:
#!/usr/bin/python

# SET I/O
inputs = {'cryption' : 'C:/Users/Nathan/Dropbox/GitHub/cryption/inputs'}
outputs = {'cryption' : 'C:/Users/Nathan/Dropbox/GitHub/cryption/outputs'}

# LIBRARIES
import os
os.chdir("C:/Users/Nathan/Dropbox/GitHub/cryption/scripts")
from X_Imports import *
script = str(os.path.realpath(__file__)) if '__file__' in globals() else 'Current Script Unknown'
header.scriptsummary(script, inputs, outputs)
pd.options.mode.chained_assignment = None
#####################################################################################################

# PARAMS
key = -6
preserve = ['\n']
sendtofile = False
inputstring = ''

# READ FILE
if (inputstring == ''):
    fn = 'input.txt'
    path = inputs['cryption'] + '/' + fn
    i = open(path, 'r')
    unencrypted_string = i.read()
    i.close()
else:
    unencrypted_string = inputstring

# TURN TO LIST
unencrypted_list = list(unencrypted_string)

# ACTUAL ENCRYPTION
nl = []
for c in unencrypted_list:
    if (c in preserve):
        nl.append(c)
    else:
        nc = ord(c)+key
        nc = unichr(nc)
        nl.append(nc)

# TURN TO STRING
encrypted_list = nl
encrypted_string = str(''.join(encrypted_list))

# WRITE FILE
if (sendtofile):
    fn = 'ue_' + fn
    path = outputs['cryption'] + '/' + fn
    o = open(path, 'w')
    o.write(encrypted_string)
    o.close()
else:
    print(encrypted_string)




######################################################

CURRENT SCRIPT:
C:\Users\Nathan\Dropbox\GitHub\cryption\scripts\X_Header.pyc

INPUT FOLDERS:
C:/Users/Nathan/Dropbox/GitHub/cryption/inputs

OUTPUT FOLDERS:
C:/Users/Nathan/Dropbox/GitHub/cryption/outputs

######################################################

We the people of the United States, in order to form a more perfect union, establish justice, insure domestic tranquility, provide for the common defense, promote the general welfare, and secure the blessings of liberty to ourselves and our posterity, do ordain and establish this Constitution for the United States of America.


In [13]:
ord('\n')

10

In [15]:
unichr(11)

u'\x0b'

In [10]:
nl

[u'u',
 u'i',
 u'f',
 u'!',
 u'r',
 u'v',
 u'j',
 u'd',
 u'l',
 u'!',
 u'c',
 u's',
 u'p',
 u'x',
 u'o',
 u'!',
 u'g',
 u'p',
 u'y',
 u'!',
 u'k',
 u'v',
 u'n',
 u'q',
 u'f',
 u'e',
 u'!',
 u'p',
 u'w',
 u'f',
 u's',
 u'!',
 u'u',
 u'i',
 u'f',
 u'!',
 u'm',
 u'b',
 u'{',
 u'z',
 u'!',
 u'e',
 u'p',
 u'h',
 u'\x0b',
 u'U',
 u'I',
 u'F',
 u'!',
 u'R',
 u'V',
 u'J',
 u'D',
 u'L',
 u'!',
 u'C',
 u'S',
 u'P',
 u'X',
 u'O',
 u'!',
 u'G',
 u'P',
 u'Y',
 u'!',
 u'K',
 u'V',
 u'N',
 u'Q',
 u'F',
 u'E',
 u'!',
 u'P',
 u'W',
 u'F',
 u'S',
 u'!',
 u'U',
 u'I',
 u'F',
 u'!',
 u'M',
 u'B',
 u'[',
 u'Z',
 u'!',
 u'E',
 u'P',
 u'H',
 u'\x0b',
 u'2',
 u'3',
 u'4',
 u'5',
 u'6',
 u'7',
 u'8',
 u'9',
 u':',
 u'1',
 u'\x0b',
 u'a',
 u'\x7f',
 u'.',
 u'>',
 u'`',
 u',',
 u'\\',
 u'^',
 u']',
 u'|',
 u'~',
 u'}',
 u'<',
 u'(',
 u';',
 u'#',
 u'-',
 u'/',
 u'0',
 u'=',
 u'?',
 u'@']